<a href="https://colab.research.google.com/github/SumathiGit/Python-World/blob/main/Auto_Encoder_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.parallel
import numpy as np
import pandas as pd
from torch.autograd import Variable

In [ ]:
# dataset source https://grouplens.org/datasets/movielens/

In [123]:
movies = pd.read_csv("data/movies.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv("data/users.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv("data/ratings.dat", sep = '::::::', header = None, engine = 'python')

In [124]:
#Preparing training and testing Datasets

training_set = pd.read_csv("data/u1.base", delimiter='\t')
training_set = np.array(training_set, dtype='int')

testing_set = pd.read_csv("data/u1.test", delimiter='\t')
testing_set = np.array(testing_set, dtype='int')


In [125]:
#Getting the number of users and movies


no_of_users = int(max(max(training_set[:,0]),max(testing_set[:,0])))
no_of_movies = int(max(max(training_set[:,1]),max(testing_set[:,1])))

In [126]:
#Converting the data ito matrix  with user and movies column
def convert(data):
    new_data = []
    for user in range(1, no_of_users+1):
        movieid = data[:,1][data[:,0] == user]
        ratingid = data[:,2][data[:,0] == user]
        ratings = np.zeros(no_of_movies)
        ratings[movieid -1] = ratingid
        new_data.append(list(ratings))
    return new_data

In [127]:
training_set = convert(training_set)
testing_set = convert(testing_set)

#So we get out training_set and testing_set in a list (2dim)

In [128]:
training_set = torch.FloatTensor(training_set)
testing_set = torch.FloatTensor(testing_set)

print(testing_set.type)

<built-in method type of Tensor object at 0x7f4a3fe6c410>


In [129]:
"""
AutoEncoder 
x >> input
f(w*x+b) = predicted values #Multiply the input with weight adding bias with a activation function we will get the pred values
Comparing the predicted values with a actual values until we get a same output as same as the input
finding the loss 
"""

'\nAutoEncoder \nx >> input\nf(w*x+b) = predicted values #Multiply the input with weight adding bias with a activation function we will get the pred values\nComparing the predicted values with a actual values until we get a same output as same as the input\nfinding the loss \n'

In [130]:
class AutoEncoder(nn.Module):
    def __init__(self, ):
        super(AutoEncoder, self).__init__()
        self.fc1 = nn.Linear(no_of_movies, 20)  #passing the users ratings (user1 , rating for all movies ..) as a input with 20 neurons in hidden layer that produce 20 output     
        self.fc2 = nn.Linear(20, 10)            #Pass the 20 outputs with 10 hidden neuron 
        self.fc3 = nn.Linear(10, 20)            #Decoding 
        self.fc4 = nn.Linear(20, no_of_movies)  #Goal is to get the input match eith the output ((no_of_movies))
        
        self.activation = nn.Sigmoid()     #we have to predict the probability as an output. Since probability of anything exists only between the range of 0 and 1 
        
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        
        return (x)

In [131]:
autoencoder = AutoEncoder()
criterion = nn.MSELoss()    # Mean squared error is calculated as the average of the squared differences between the predicted and actual values. 
optimizer = torch.optim.RMSprop(autoencoder.parameters(), lr = 0.01, weight_decay= 0.5) #root mean square propagation 

In [132]:
#Train the autoencoder
epochs = 200
for epoch in range(1, epochs +1):      
    for user in range(no_of_users):
        input = Variable(training_set[user]).unsqueeze(0)  #unsqeeze will add an extra column in the training_set ==> single dim)>> Variable need multi dim data
        target = input.clone()
        if torch.sum(target.data > 0 ) > 0 :
          output =autoencoder(input)
          loss = criterion(output, target)
          loss.backward()
          optimizer.step()                                  #Step to adjust ypur weight in each iteration
    print("Epoch: " +str(epoch)+" loss:" +str(loss.data))
    

Epoch: 1 loss:tensor(1.0077)
Epoch: 2 loss:tensor(0.9062)
Epoch: 3 loss:tensor(0.8904)
Epoch: 4 loss:tensor(0.8854)
Epoch: 5 loss:tensor(0.9080)
Epoch: 6 loss:tensor(0.8847)
Epoch: 7 loss:tensor(0.8878)
Epoch: 8 loss:tensor(0.8895)
Epoch: 9 loss:tensor(0.8836)
Epoch: 10 loss:tensor(0.8808)
Epoch: 11 loss:tensor(0.9064)
Epoch: 12 loss:tensor(0.8844)
Epoch: 13 loss:tensor(0.8987)
Epoch: 14 loss:tensor(0.8812)
Epoch: 15 loss:tensor(0.8808)
Epoch: 16 loss:tensor(0.8985)
Epoch: 17 loss:tensor(0.8950)
Epoch: 18 loss:tensor(0.8795)
Epoch: 19 loss:tensor(0.8842)
Epoch: 20 loss:tensor(0.8814)
Epoch: 21 loss:tensor(0.9200)
Epoch: 22 loss:tensor(0.8981)
Epoch: 23 loss:tensor(0.9287)
Epoch: 24 loss:tensor(0.8813)
Epoch: 25 loss:tensor(0.8848)
Epoch: 26 loss:tensor(0.9169)
Epoch: 27 loss:tensor(0.8780)
Epoch: 28 loss:tensor(0.8822)
Epoch: 29 loss:tensor(0.9272)
Epoch: 30 loss:tensor(0.8978)
Epoch: 31 loss:tensor(0.8843)
Epoch: 32 loss:tensor(0.8811)
Epoch: 33 loss:tensor(0.8785)
Epoch: 34 loss:tens

In [132]:
#Initially the loss is 1 after that it gradually decreased to 0.79

In [ ]:
#Testing the trained autoencoder using testing_set to check whether the loss is minimized or not

In [134]:
total = 0
for user in range(no_of_users):
        input = Variable(training_set[user]).unsqueeze(0)  
        target = Variable(testing_set[user])      #Passing the testing_set
        if torch.sum(target.data > 0 ) > 0 :
          output =autoencoder(input)
          loss = criterion(output, target)                                 
          print(" loss:" +str(loss.data))
    

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


 loss:tensor(0.9525)
 loss:tensor(0.1882)
 loss:tensor(0.1100)
 loss:tensor(0.1121)
 loss:tensor(0.4907)
 loss:tensor(0.6513)
 loss:tensor(1.4954)
 loss:tensor(0.2651)
 loss:tensor(0.1293)
 loss:tensor(0.7980)
 loss:tensor(0.6004)
 loss:tensor(0.2919)
 loss:tensor(1.8375)
 loss:tensor(0.4742)
 loss:tensor(0.2665)
 loss:tensor(0.7259)
 loss:tensor(0.0674)
 loss:tensor(0.8812)
 loss:tensor(0.0862)
 loss:tensor(0.1826)
 loss:tensor(0.4066)
 loss:tensor(0.5024)
 loss:tensor(0.5235)
 loss:tensor(0.3659)
 loss:tensor(0.4384)
 loss:tensor(0.2078)
 loss:tensor(0.1001)
 loss:tensor(0.3097)
 loss:tensor(0.1160)
 loss:tensor(0.1473)
 loss:tensor(0.1340)
 loss:tensor(0.1059)
 loss:tensor(0.0819)
 loss:tensor(0.1136)
 loss:tensor(0.0838)
 loss:tensor(0.1044)
 loss:tensor(0.2386)
 loss:tensor(0.4879)
 loss:tensor(0.0790)
 loss:tensor(0.0853)
 loss:tensor(0.2046)
 loss:tensor(0.6608)
 loss:tensor(0.7399)
 loss:tensor(0.5823)
 loss:tensor(0.1433)
 loss:tensor(0.1479)
 loss:tensor(0.0943)
 loss:tensor(